# NIfTI Volume Resampling: 100μm Isotropic → 100×100×150μm

This notebook demonstrates how to resample an MRI volume from isotropic 100 micron spacing to anisotropic spacing (100×100×150 microns).

In [ ]:
# Install required packages if needed
# !pip install nibabel numpy scipy matplotlib --break-system-packages

In [ ]:
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from pathlib import Path

## Method 1: Simple Resampling Function

In [ ]:
def resample_z_spacing(nifti_img, new_z_spacing_mm=0.15, order=1):
    """
    Resample only the z-direction of a NIfTI image.
    
    Parameters:
    -----------
    nifti_img : nibabel.Nifti1Image
        Input NIfTI image
    new_z_spacing_mm : float
        New z-spacing in mm (0.15 = 150 microns)
    order : int
        Interpolation order (0=nearest, 1=linear, 3=cubic)
    
    Returns:
    --------
    nibabel.Nifti1Image
        Resampled image
    """
    # Get data and affine
    data = nifti_img.get_fdata()
    affine = nifti_img.affine.copy()
    
    # Get current z-spacing
    current_z_spacing = np.abs(affine[2, 2])
    
    # Calculate zoom factor for z-direction
    z_zoom = current_z_spacing / new_z_spacing_mm
    
    print(f"Original shape: {data.shape}")
    print(f"Original z-spacing: {current_z_spacing*1000:.1f} μm")
    print(f"Target z-spacing: {new_z_spacing_mm*1000:.1f} μm")
    print(f"Z-axis zoom factor: {z_zoom:.4f}")
    
    # Resample (only z-axis changes)
    zoom_factors = [1.0, 1.0, z_zoom]
    resampled_data = zoom(data, zoom_factors, order=order, mode='nearest')
    
    print(f"Resampled shape: {resampled_data.shape}")
    print(f"Z slices: {data.shape[2]} → {resampled_data.shape[2]}")
    
    # Update affine matrix
    new_affine = affine.copy()
    new_affine[2, 2] = new_z_spacing_mm if affine[2, 2] > 0 else -new_z_spacing_mm
    
    # Create new NIfTI image
    resampled_img = nib.Nifti1Image(resampled_data, new_affine, nifti_img.header)
    
    return resampled_img

## Example Usage

In [ ]:
# Load your NIfTI file
input_path = 'your_mri_volume.nii.gz'  # Replace with your file path

# Load the image
nifti_img = nib.load(input_path)

print("Original volume information:")
print(f"Shape: {nifti_img.shape}")
print(f"Affine matrix:\n{nifti_img.affine}")
print(f"Voxel dimensions (pixdim): {nifti_img.header['pixdim'][1:4]}")

In [ ]:
# Resample with different interpolation methods
print("\n" + "="*60)
print("LINEAR INTERPOLATION (recommended)")
print("="*60)
resampled_linear = resample_z_spacing(nifti_img, new_z_spacing_mm=0.15, order=1)

print("\n" + "="*60)
print("NEAREST NEIGHBOR INTERPOLATION")
print("="*60)
resampled_nearest = resample_z_spacing(nifti_img, new_z_spacing_mm=0.15, order=0)

print("\n" + "="*60)
print("CUBIC INTERPOLATION")
print("="*60)
resampled_cubic = resample_z_spacing(nifti_img, new_z_spacing_mm=0.15, order=3)

## Visualization: Compare Interpolation Methods

In [ ]:
def visualize_slice_comparison(original, nearest, linear, cubic, slice_idx=None):
    """
    Visualize a central slice from original and resampled volumes.
    """
    # Get data
    orig_data = original.get_fdata()
    
    # If no slice specified, use middle slice
    if slice_idx is None:
        slice_idx = orig_data.shape[2] // 2
    
    # Calculate corresponding slice in resampled volume
    resample_slice_idx = int(slice_idx * nearest.shape[2] / orig_data.shape[2])
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    
    # Original
    axes[0, 0].imshow(orig_data[:, :, slice_idx].T, cmap='gray', origin='lower')
    axes[0, 0].set_title(f'Original (100μm iso)\nSlice {slice_idx}/{orig_data.shape[2]}')
    axes[0, 0].axis('off')
    
    # Nearest neighbor
    near_data = nearest.get_fdata()
    axes[0, 1].imshow(near_data[:, :, resample_slice_idx].T, cmap='gray', origin='lower')
    axes[0, 1].set_title(f'Nearest Neighbor\nSlice {resample_slice_idx}/{near_data.shape[2]}')
    axes[0, 1].axis('off')
    
    # Linear
    lin_data = linear.get_fdata()
    axes[1, 0].imshow(lin_data[:, :, resample_slice_idx].T, cmap='gray', origin='lower')
    axes[1, 0].set_title(f'Linear (Trilinear)\nSlice {resample_slice_idx}/{lin_data.shape[2]}')
    axes[1, 0].axis('off')
    
    # Cubic
    cub_data = cubic.get_fdata()
    axes[1, 1].imshow(cub_data[:, :, resample_slice_idx].T, cmap='gray', origin='lower')
    axes[1, 1].set_title(f'Cubic\nSlice {resample_slice_idx}/{cub_data.shape[2]}')
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize middle slice
visualize_slice_comparison(nifti_img, resampled_nearest, resampled_linear, resampled_cubic)

## Visualize Z-axis Profile (Sagittal/Coronal View)

In [ ]:
def visualize_z_comparison(original, linear, x_idx=None, y_idx=None):
    """
    Show sagittal or coronal view to see z-direction resampling effect.
    """
    orig_data = original.get_fdata()
    lin_data = linear.get_fdata()
    
    # Use center if not specified
    if x_idx is None:
        x_idx = orig_data.shape[0] // 2
    if y_idx is None:
        y_idx = orig_data.shape[1] // 2
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Sagittal view (YZ plane)
    axes[0].imshow(orig_data[x_idx, :, :].T, cmap='gray', 
                   aspect=1.0, origin='lower')
    axes[0].set_title(f'Original Sagittal View\nShape: {orig_data.shape[1]} × {orig_data.shape[2]}')
    axes[0].set_xlabel('Y axis')
    axes[0].set_ylabel('Z axis (100 μm spacing)')
    
    axes[1].imshow(lin_data[x_idx, :, :].T, cmap='gray',
                   aspect=1.5, origin='lower')  # aspect ratio adjusted for new spacing
    axes[1].set_title(f'Resampled Sagittal View\nShape: {lin_data.shape[1]} × {lin_data.shape[2]}')
    axes[1].set_xlabel('Y axis')
    axes[1].set_ylabel('Z axis (150 μm spacing)')
    
    plt.tight_layout()
    plt.show()

visualize_z_comparison(nifti_img, resampled_linear)

## Quantitative Comparison

In [ ]:
def compare_statistics(original, resampled, method_name):
    """
    Compare intensity statistics between original and resampled volumes.
    """
    orig_data = original.get_fdata()
    resamp_data = resampled.get_fdata()
    
    print(f"\n{method_name} Statistics:")
    print("-" * 50)
    print(f"Original - Min: {orig_data.min():.2f}, Max: {orig_data.max():.2f}, "
          f"Mean: {orig_data.mean():.2f}, Std: {orig_data.std():.2f}")
    print(f"Resampled - Min: {resamp_data.min():.2f}, Max: {resamp_data.max():.2f}, "
          f"Mean: {resamp_data.mean():.2f}, Std: {resamp_data.std():.2f}")
    print(f"Mean difference: {abs(orig_data.mean() - resamp_data.mean()):.4f}")
    print(f"Std difference: {abs(orig_data.std() - resamp_data.std()):.4f}")

compare_statistics(nifti_img, resampled_nearest, "Nearest Neighbor")
compare_statistics(nifti_img, resampled_linear, "Linear")
compare_statistics(nifti_img, resampled_cubic, "Cubic")

## Save Resampled Volume

In [ ]:
# Save the resampled volume (using linear interpolation)
output_path = 'resampled_volume_150um.nii.gz'
nib.save(resampled_linear, output_path)
print(f"Saved resampled volume to: {output_path}")

# Verify the saved file
saved_img = nib.load(output_path)
print(f"\nVerification:")
print(f"Shape: {saved_img.shape}")
print(f"Voxel dims: {saved_img.header['pixdim'][1:4]} mm")
print(f"Voxel dims: {saved_img.header['pixdim'][1:4] * 1000} μm")

## Alternative: Using SimpleITK (More Robust)

SimpleITK provides more sophisticated resampling with better handling of orientation and spacing.

In [ ]:
# Uncomment to install SimpleITK
# !pip install SimpleITK --break-system-packages

import SimpleITK as sitk

def resample_sitk(input_path, output_path, new_spacing=(0.1, 0.1, 0.15), interpolator='linear'):
    """
    Resample using SimpleITK (more robust for medical images).
    
    Parameters:
    -----------
    input_path : str
        Input NIfTI file
    output_path : str
        Output NIfTI file
    new_spacing : tuple
        (x, y, z) spacing in mm
    interpolator : str
        'linear', 'nearest', 'bspline', or 'cubic'
    """
    # Read image
    image = sitk.ReadImage(input_path)
    
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    
    print(f"Original spacing: {original_spacing}")
    print(f"Original size: {original_size}")
    
    # Calculate new size
    new_size = [
        int(round(original_size[0] * (original_spacing[0] / new_spacing[0]))),
        int(round(original_size[1] * (original_spacing[1] / new_spacing[1]))),
        int(round(original_size[2] * (original_spacing[2] / new_spacing[2])))
    ]
    
    print(f"New spacing: {new_spacing}")
    print(f"New size: {new_size}")
    
    # Set interpolator
    interpolator_map = {
        'nearest': sitk.sitkNearestNeighbor,
        'linear': sitk.sitkLinear,
        'bspline': sitk.sitkBSpline,
        'cubic': sitk.sitkBSpline
    }
    
    # Resample
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetDefaultPixelValue(0)
    resampler.SetInterpolator(interpolator_map[interpolator])
    
    resampled = resampler.Execute(image)
    
    # Save
    sitk.WriteImage(resampled, output_path)
    print(f"Saved to: {output_path}")
    
    return resampled

# Example usage with SimpleITK
# resampled_sitk = resample_sitk(
#     'your_mri_volume.nii.gz',
#     'resampled_sitk_150um.nii.gz',
#     new_spacing=(0.1, 0.1, 0.15),  # in mm
#     interpolator='linear'
# )